In [1]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('data/train.csv')
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [55]:
# customer_country, customer_country.1 이 같음 하나 삭제
df.drop(columns=['customer_country.1'], inplace= True)
df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']] = df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']].fillna(0)
print(df.shape)
df.isnull().sum()

(59299, 28)


bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver               0
it_strategic_ver               0
idit_strategic_ver             0
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

In [73]:
df['customer_type'].value_counts()

End-Customer                    6647
End Customer                    3996
Specifier/ Influencer           2525
Channel Partner                 1368
Service Partner                  349
Solution Eco-Partner             146
Installer/Contractor              52
Specifier / Influencer            43
Corporate                         31
HVAC Engineer                     23
Engineer                          20
Developer                         17
Technician                        16
Consultant                        15
Other                             10
Home Owner                        10
End-user                           8
Manager / Director                 8
Software/Solution Provider         7
Etc.                               6
Architect/Consultant               5
Homeowner                          5
Reseller                           5
Installer                          5
Interior Designer                  5
Distributor                        4
Others                             4
D

In [72]:
df['is_converted'].value_counts()

False    54449
True      4850
Name: is_converted, dtype: int64

In [65]:
df['customer_job'].value_counts().head(20)

engineering                       6342
other                             4852
administrative                    3359
education                         2287
sales                             2218
operations                        1961
purchasing                        1938
information technology            1937
business development              1801
consulting                        1349
arts and design                   1276
entrepreneurship                  1192
marketing                         1161
program and project management     948
others                             763
media and communication            709
healthcare services                632
product management                 573
finance                            463
accounting                         396
Name: customer_job, dtype: int64

In [71]:
df[df['customer_job'] == 'sales']['is_converted'].value_counts()

False    2077
True      141
Name: is_converted, dtype: int64

In [62]:
df[df['historical_existing_cnt'] == df['historical_existing_cnt'].max()]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
26689,0.5,/Honolulu/United States,AS,NaN,12698,End-Customer,Enterprise,1394.0,0.0,0.0,...,LGEUS,NaN,1,0,0.001183,0.011583,retail,NaN,27,False


In [59]:
df[['customer_idx', 'historical_existing_cnt']][df['customer_idx'] == 23122]

,customer_idx,historical_existing_cnt
1,23122,12.0
13622,23122,12.0
13623,23122,12.0
57574,23122,NaN


In [58]:
df[['customer_idx', 'historical_existing_cnt']].groupby('customer_idx').

,customer_idx,historical_existing_cnt
0,32160,NaN
1,23122,12.0
2,1755,144.0
3,4919,NaN
4,17126,NaN
...,...,...
59294,33747,NaN
59295,35420,NaN
59296,19249,NaN
59297,40327,NaN


In [15]:
df[['customer_idx', 'is_converted']].groupby('customer_idx').sum().sort_values(by = 'is_converted', ascending= False).head(20)

,is_converted
customer_idx,
25096,2421
47466,67
14746,38
42108,19
7740,17
45319,15
9324,14
26337,13
9857,11


In [36]:
df['customer_idx'].value_counts().head(20)

47466    7686
25096    2421
37680     367
18030     331
21321     162
33350     135
43395      91
25309      90
33763      88
32240      84
37654      76
31864      71
37657      67
19804      65
29370      61
40344      59
21389      51
7195       50
37399      45
42067      43
Name: customer_idx, dtype: int64

In [34]:
df[df['customer_idx'] == 13580]['is_converted'].value_counts()

True    6
Name: is_converted, dtype: int64

In [4]:
df['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [33]:
df['customer_idx'].value_counts() > 1

customer_idx
47466     True
25096     True
37680     True
18030     True
21321     True
         ...  
39487    False
41241    False
39422    False
22515    False
30268    False
Name: count, Length: 35112, dtype: bool

In [32]:
df[df['customer_idx']== 21321]['is_converted'].value_counts()

is_converted
False    161
True       1
Name: count, dtype: int64

In [16]:
temp = pd.DataFrame(df['customer_country'])
temp['country'] = temp['customer_country'].apply(lambda x : str(x).split('/')[-1])
temp['country'].unique()

array(['Philippines', 'India', 'Nigeria', 'Saudi Arabia', 'Singapore',
       'Brazil', 'U.A.E', 'South Africa', 'United States', 'Colombia',
       'Mexico', 'Ghana', 'Egypt', 'Democratic Republic of the Congo',
       'Ethiopia', 'Australia', 'nan', 'Kenya', 'Indonesia', 'Oman',
       'Pakistan', 'United Kingdom', 'Guatemala', 'Panama', 'Canada',
       'Bangladesh', 'Papua New Guinea', 'United Republic of Tanzania',
       'Qatar', 'Afghanistan', 'Chile', 'Mozambique', 'Turkey', 'Türkiye',
       'El Salvador', 'Togo', 'Jordan', 'Iraq', 'Israel', 'Sri Lanka',
       'South Korea', 'Portugal', 'Mauritania', 'Uruguay', 'Peru',
       'Germany', 'Romania', 'Norway', 'Jamaica', '', 'Hungary',
       ' Connecticut United States', 'Poland', 'Czech', 'Spain',
       'Argentina', 'Ecuador', 'Senegal', 'Hong Kong', 'Malaysia',
       'Japan', 'Kuwait', 'Ireland', 'Albania', 'Greece', 'Barranquilla',
       'Algeria', 'ARGENTINA', 'Nicaragua', 'Slovenia', 'Italy',
       'Netherlands', 'Domi

In [21]:
A = 'EPYPT'
A.split('/')[-1]

'EPYPT'

In [9]:
df['bant_submit'].unique()

array([1.  , 0.75, 0.5 , 0.25, 0.  ])

In [14]:
df['customer_country'].apply(lambda x : str(x).split('/')[-1])

0        Philippines
1        Philippines
2              India
3              India
4              India
            ...     
59294         Poland
59295       Colombia
59296           Peru
59297           Peru
59298         France
Name: customer_country, Length: 59299, dtype: object